In [1]:
import torch
import os
import matplotlib.pyplot as plt
os.chdir("..")

from epilearn.models.SpatialTemporal.STGCN import STGCN
from epilearn.models.SpatialTemporal.MepoGNN import MepoGNN
from epilearn.models.SpatialTemporal.EpiGNN import EpiGNN
from epilearn.models.SpatialTemporal.DASTGN import DASTGN
from epilearn.models.SpatialTemporal.ColaGNN import ColaGNN
from epilearn.models.SpatialTemporal.EpiColaGNN import EpiColaGNN
from epilearn.models.SpatialTemporal.CNNRNN_Res import CNNRNN_Res
from epilearn.models.SpatialTemporal.ATMGNN import MPNN_LSTM, ATMGNN

from epilearn.data import UniversalDataset
from epilearn.utils import utils, transforms
from epilearn.tasks.forecast import Forecast

### Configs

In [4]:
# initial settings
device = torch.device('cpu')
torch.manual_seed(7)

lookback = 12 # inputs size
horizon = 3 # predicts size

# permutation is True when using STGCN
permute = False

epochs = 50 # training epochs
batch_size = 50 # training batch size

### Initialize dataset

In [3]:
# load toy dataset
dataset = UniversalDataset()
dataset.load_toy_dataset()

### Initialize model and task

In [7]:
task = Forecast(prototype=EpiGNN, dataset=None, lookback=lookback, horizon=horizon, device='cpu')

### Add transforms to dataset

In [ ]:
# transforms = transforms.Compose({"features":[], 'graph': [], 'states': []})

### Train Model

In [8]:
config = None
# for epicolagnn, loss='epi_cola', else loss='mse
# for STGCN, permute_dataset=True
model = task.train_model(dataset=dataset, config=config, loss='mse', epochs=5, permute_dataset=False) # instead of config, we can also dircetly input some parameters

spatial-temporal model loaded!


 20%|██        | 1/5 [00:00<00:01,  3.32it/s]

######### epoch:0
Training loss: 149.76375678277785
Validation loss: 1.1111464500427246


100%|██████████| 5/5 [00:01<00:00,  4.05it/s]



Final Training loss: 17.058270331351988
Final Validation loss: 0.5010042786598206
Test MAE: 204.12928771972656
Test RMSE: 535.9934692382812


### Evaluate model

In [ ]:
# evaluation = task.evaluate_model(dataset, config=config)